In [1]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx

In [2]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/sax.wav"

In [3]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([ 0.0040,  0.0078,  0.0093,  ..., -0.0001, -0.0001, -0.0001])


In [4]:
audio.shape

torch.Size([28809])

In [6]:
rfft_audio.apply_rfft(audio).shape

torch.Size([14405])

In [7]:
class RFFTModel(nn.Module):
    def __init__(self):
        super(RFFTModel,self).__init__()
    
    def forward(self, audio):
        rfft_magnitude = rfft_audio.apply_rfft(audio)
        return rfft_magnitude

In [8]:
model = RFFTModel()

In [9]:
export_option = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx_program = torch.onnx.dynamo_export(
    model,
    audio,
)

/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/satya-tt0492/anaconda3/envs/ex5/lib/python3.9/site-packages/torch/onnx/_internal/fx/onnxfunction_dispatcher.py:529:

In [10]:
onnx_program.save("rfft_model.onnx")

In [11]:
onnx_model = onnx.load("rfft_model.onnx")
onnx.checker.check_model(onnx_model)

In [12]:
import onnxruntime

In [13]:
ort_session = onnxruntime.InferenceSession("rfft_model.onnx")

In [14]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [15]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(audio)

In [16]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnx_output = ort_session.run(None,onnxruntime_input)

In [20]:
onnx_output[0][:10]

array([ 1.5710768, 12.565786 , 35.97016  , 10.405327 , 36.515358 ,
       36.303947 , 18.488478 , 19.208668 , 36.191345 , 22.74525  ],
      dtype=float32)

In [18]:
rfft_audio.apply_rfft(audio)[1]

tensor(12.5656)